# AutoGen মৌলিক নমুনা

এই কোড নমুনায়, আপনি [AutoGen](https://aka.ms/ai-agents/autogen) AI ফ্রেমওয়ার্ক ব্যবহার করে একটি মৌলিক এজেন্ট তৈরি করবেন।

এই নমুনার লক্ষ্য হলো আপনাকে সেই ধাপগুলো দেখানো, যা আমরা পরে বিভিন্ন এজেন্টিক প্যাটার্ন বাস্তবায়নের সময় অতিরিক্ত কোড নমুনাগুলোতে ব্যবহার করব।


## প্রয়োজনীয় পাইথন প্যাকেজগুলি ইমপোর্ট করুন


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## ক্লায়েন্ট তৈরি করুন

এই উদাহরণে, আমরা [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ব্যবহার করব LLM-এ অ্যাক্সেসের জন্য।

`model` হিসেবে `gpt-4o-mini` নির্ধারণ করা হয়েছে। GitHub Models মার্কেটপ্লেসে উপলব্ধ অন্য কোনো মডেলে পরিবর্তন করে দেখুন বিভিন্ন ফলাফল কেমন হয়।

একটি দ্রুত পরীক্ষা হিসেবে, আমরা একটি সাধারণ প্রম্পট চালাব - `What is the capital of France`।


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## এজেন্ট সংজ্ঞায়িত করা

এখন যেহেতু আমরা `client` সেটআপ করেছি এবং নিশ্চিত করেছি যে এটি কাজ করছে, চলুন একটি `AssistantAgent` তৈরি করি। প্রতিটি এজেন্টকে নিম্নলিখিত বিষয়গুলো নির্ধারণ করা যেতে পারে:
**name** - একটি সংক্ষিপ্ত নাম যা বহু-এজেন্ট প্রবাহে এটি উল্লেখ করার জন্য কার্যকর হবে।
**model_client** - পূর্ববর্তী ধাপে আপনি যে ক্লায়েন্ট তৈরি করেছেন।
**tools** - এজেন্টের জন্য উপলব্ধ টুলস যা একটি কাজ সম্পন্ন করতে ব্যবহার করা যেতে পারে।
**system_message** - মেটাপ্রম্পট যা LLM-এর কাজ, আচরণ এবং টোন সংজ্ঞায়িত করে।

আপনি `system_message` পরিবর্তন করতে পারেন এবং দেখতে পারেন LLM কীভাবে প্রতিক্রিয়া জানায়। আমরা `tools` নিয়ে আলোচনা করব পাঠ #৪-এ।


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## এজেন্ট চালান

নিচের ফাংশনটি এজেন্ট চালানোর জন্য ব্যবহৃত হবে। আমরা `on_message` মেথডটি ব্যবহার করে এজেন্টের স্টেট নতুন বার্তার মাধ্যমে আপডেট করি।

এই ক্ষেত্রে, আমরা ব্যবহারকারীর কাছ থেকে একটি নতুন বার্তা দিয়ে স্টেট আপডেট করি, যা হলো `"Plan me a great sunny vacation"`।

আপনি বার্তার বিষয়বস্তু পরিবর্তন করে দেখতে পারেন কীভাবে LLM ভিন্নভাবে প্রতিক্রিয়া জানায়।


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় থাকা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হয়। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
